**FER in CK+ Dataset Implementation**

In this experiment, aiming to complement our models performance on primary dataset, we seek to evaluate them on this higher-quality CK+ dataset.

**Data Uploading:**
CK+ file ('ckextended.csv') includes facial expression data of emotion labels and pixel values.

In [ ]:
from google.colab import files
uploaded = files.upload()

#load
import pandas as pd
df = pd.read_csv("ckextended.csv")
print(df.columns)
print(df.head())

#data splitting
train_df = df[df['Usage'] == 'Training']
val_df   = df[df['Usage'] == 'PublicTest']
test_df  = df[df['Usage'] == 'PrivateTest']

Saving ckextended.csv to ckextended (1).csv
Index(['emotion', 'pixels', 'Usage'], dtype='object')
   emotion                                             pixels     Usage
0        6  36 39 35 25 19 11 8 7 3 13 15 9 21 57 75 90 10...  Training
1        6  88 74 19 4 5 5 3 12 8 21 15 21 15 18 24 29 32 ...  Training
2        6  9 2 4 7 1 1 1 0 7 29 49 76 115 141 156 169 177...  Training
3        6  104 106 108 104 95 50 60 61 58 83 126 133 139 ...  Training
4        6  68 72 67 67 6 2 1 1 1 1 1 14 24 24 38 65 79 94...  Training


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

class FERDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform

    def __getitem__(self, idx):
        pixels = self.df.loc[idx, 'pixels']
        img = np.fromstring(pixels, sep=' ', dtype=np.uint8).reshape(48, 48)
        img = Image.fromarray(img, mode='L')
        label = int(self.df.loc[idx, 'emotion'])
        if self.transform:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.df)

Data Preprocessing

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((48, 48)),#
    transforms.ToTensor()
])

train_dataset = FERDataset(train_df, transform=transform)
val_dataset   = FERDataset(val_df, transform=transform)
test_dataset  = FERDataset(test_df, transform=transform)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)
test_loader  = DataLoader(test_dataset, batch_size=batch_size)

print(f" Loaded {len(train_dataset)} train, {len(val_dataset)} val, {len(test_dataset)} test samples.")

 Loaded 734 train, 91 val, 95 test samples.


CK+ on TinyCNN

In [ ]:
import torch
import torch.nn as nn
import random
import numpy as np

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using", device)

class TinyCNN(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(3),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

Using cuda


In [ ]:
def train_one_epoch(model, loader, loss_fn, optim, device):
    model.train()
    running = 0.0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optim.zero_grad()
        logits = model(x)
        loss = loss_fn(logits, y)
        loss.backward()
        optim.step()
        running += loss.item() * x.size(0)
    return running / len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader, loss_fn, device):
    model.eval()
    running, correct = 0.0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        running += loss_fn(logits, y).item() * x.size(0)
        correct += (logits.argmax(1) == y).sum().item()
    return running / len(loader.dataset), correct / len(loader.dataset)

In [ ]:
model = TinyCNN(num_classes=8).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=1e-4)

best_acc = 0.0
for epoch in range(1, 21):
    train_loss = train_one_epoch(model, train_loader, loss_fn, optimizer, device)
    val_loss, val_acc = evaluate(model, val_loader, loss_fn, device)
    print(f"[Epoch {epoch:02d}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2%}")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "tinycnn_best.pth")

print("Training complete. Best Val Accuracy:", best_acc)
# Load the best model
model.load_state_dict(torch.load("tinycnn_best.pth"))
model.eval()

# Evaluate on test set
test_loss, test_acc = evaluate(model, test_loader, loss_fn, device)
print(f"Final Test Accuracy: {test_acc:.2%}")

[Epoch 01] Train Loss: 1.4966 | Val Loss: 1.8235 | Val Acc: 64.84%
[Epoch 02] Train Loss: 1.2441 | Val Loss: 1.5920 | Val Acc: 64.84%
[Epoch 03] Train Loss: 1.1535 | Val Loss: 1.3268 | Val Acc: 64.84%
[Epoch 04] Train Loss: 1.0353 | Val Loss: 1.1066 | Val Acc: 64.84%
[Epoch 05] Train Loss: 0.9282 | Val Loss: 0.9605 | Val Acc: 70.33%
[Epoch 06] Train Loss: 0.7926 | Val Loss: 0.8525 | Val Acc: 73.63%
[Epoch 07] Train Loss: 0.6929 | Val Loss: 0.7687 | Val Acc: 79.12%
[Epoch 08] Train Loss: 0.6480 | Val Loss: 0.7202 | Val Acc: 79.12%
[Epoch 09] Train Loss: 0.5734 | Val Loss: 0.6758 | Val Acc: 80.22%
[Epoch 10] Train Loss: 0.5097 | Val Loss: 0.6407 | Val Acc: 85.71%
[Epoch 11] Train Loss: 0.4417 | Val Loss: 0.6045 | Val Acc: 86.81%
[Epoch 12] Train Loss: 0.4014 | Val Loss: 0.5890 | Val Acc: 82.42%
[Epoch 13] Train Loss: 0.3612 | Val Loss: 0.5684 | Val Acc: 85.71%
[Epoch 14] Train Loss: 0.3390 | Val Loss: 0.5897 | Val Acc: 83.52%
[Epoch 15] Train Loss: 0.3065 | Val Loss: 0.5226 | Val Acc: 87

The final test accuracy reached 83.16%, with the CK+ dataset the model shows a good generalization ability. Train losses continues to decrease, showing this is an efficient learning progress and no overfitting.

CK+ on MobileNet: we varied data preprocessing pipline to fit different channels requirement (from 1 channel to 3 channels).

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])

train_dataset = FERDataset(train_df, transform=transform)
val_dataset   = FERDataset(val_df, transform=transform)
test_dataset  = FERDataset(test_df, transform=transform)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)
test_loader  = DataLoader(test_dataset, batch_size=batch_size)

print(f" Loaded {len(train_dataset)} train, {len(val_dataset)} val, {len(test_dataset)} test samples.")

 Loaded 734 train, 91 val, 95 test samples.


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# MobileNetV2 setup
mobilenet = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
mobilenet.classifier[1] = nn.Linear(mobilenet.last_channel, 8)  # 7 emotion classes
mobilenet = mobilenet.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mobilenet.parameters(), lr=1e-4)

# Training loop (short version)
best_acc = 0
for epoch in range(1, 21):
    train_loss = train_one_epoch(mobilenet, train_loader, loss_fn, optimizer, device)
    val_loss, val_acc = evaluate(mobilenet, val_loader, loss_fn, device)
    print(f"[MobileNetV2] Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.2%}")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(mobilenet.state_dict(), "mobilenetv2_ckplus_best.pth")

# Final test accuracy
test_loss, test_acc = evaluate(mobilenet, test_loader, loss_fn, device)
print(f" MobileNetV2 Test Accuracy: {test_acc:.2%}")

[MobileNetV2] Epoch 01 | Train Loss: 1.9297 | Val Acc: 64.84%
[MobileNetV2] Epoch 02 | Train Loss: 1.5510 | Val Acc: 64.84%
[MobileNetV2] Epoch 03 | Train Loss: 1.2166 | Val Acc: 64.84%
[MobileNetV2] Epoch 04 | Train Loss: 0.9425 | Val Acc: 64.84%
[MobileNetV2] Epoch 05 | Train Loss: 0.7149 | Val Acc: 68.13%
[MobileNetV2] Epoch 06 | Train Loss: 0.5557 | Val Acc: 74.73%
[MobileNetV2] Epoch 07 | Train Loss: 0.4179 | Val Acc: 82.42%
[MobileNetV2] Epoch 08 | Train Loss: 0.3261 | Val Acc: 82.42%
[MobileNetV2] Epoch 09 | Train Loss: 0.2703 | Val Acc: 83.52%
[MobileNetV2] Epoch 10 | Train Loss: 0.2163 | Val Acc: 82.42%
[MobileNetV2] Epoch 11 | Train Loss: 0.1731 | Val Acc: 83.52%
[MobileNetV2] Epoch 12 | Train Loss: 0.1426 | Val Acc: 83.52%
[MobileNetV2] Epoch 13 | Train Loss: 0.1127 | Val Acc: 83.52%
[MobileNetV2] Epoch 14 | Train Loss: 0.0857 | Val Acc: 82.42%
[MobileNetV2] Epoch 15 | Train Loss: 0.0669 | Val Acc: 82.42%
[MobileNetV2] Epoch 16 | Train Loss: 0.0453 | Val Acc: 82.42%
[MobileN

After the same 20 epochs of trainig, the MobileNet reached a test accuracy of 85.26%, slightly better than TinyCNN's results. The model reached a stable validation, but the train loss keep decreases, no overfitting happens.

**Conclusion:** With CK+, models achieved better performances. TinyCNN achieves from 29.54% (in primary FER dataset) to 83.16% and MobileNetV2 rises from 65.38% to 85.26%. This experiment's signifance is showing effective dataset choosing and preprocessing's impact in evaluating facial emotion data.